<a href="https://colab.research.google.com/github/zenikigai/Pengembangan_Machine_Learning_IDcamp2023/blob/main/multiclass_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Text Classification With LSTM

In this exercise, we will classify multiclass texts using LSTM.

In this exercise we will use a dataset containing synopses of several Indonesian films and their genres. Our goal is to determine the genre of a movie based on its synopsis.

we will use this dtataset, you can download it here: https://www.kaggle.com/datasets/antoniuscs/imdb-synopsis-indonesian-movies

In the first cell, import the pandas library and convert the dataset into a dataframe. Then discard the 'judul_film' column as we'll just be using the synopsis as an attribute to train on the model.

In [1]:
import pandas as pd
df = pd.read_csv("imdb_indonesian_movies_2.csv")
df = df.drop(columns=["judul_film"])

In [2]:
# show the top 5 columns on dataset
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Since our labels are categorical data, we need to do a one-hot-encoding process. Run the code below to perform a one-hot-encoding and create a new dataframe.

In [3]:
category = pd.get_dummies(df.genre)
new_df = pd.concat([df, category], axis=1)
new_df = new_df.drop(columns="genre")
new_df

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In order for the model to be processed by the model, we need to convert the values from the dataframe into numpy array data types using the values attribute.

In [5]:
sinopsys = new_df["ringkasan_sinopsis"].values
label = new_df[["Drama", "Horor", "Komedi", "Laga", "Romantis"]].values

In [6]:
# divide data into training and test
from sklearn.model_selection import train_test_split
sinopsys_train, sinopsys_test, label_train, label_test = train_test_split(sinopsys, label, test_size=0.2)

Then we convert every word in our dataset into a numeric number with the Tokenizer function. Once tokenization is complete, we need to make converting each sample into a sequence.

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="x")
tokenizer.fit_on_texts(sinopsys_train)
tokenizer.fit_on_texts(sinopsys_test)

sequence_train = tokenizer.texts_to_sequences(sinopsys_train)
sequence_test = tokenizer.texts_to_sequences(sinopsys_test)

padded_train = pad_sequences(sequence_train)
padded_test = pad_sequences(sequence_test)

For the model architecture we use an Embedding layer with an embedding dimension of 16, as well as an input dimension of num_words value on the tokenizer object. Don't forget to call the compile function and specify the optimizer and loss function that will be used by the model.

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

model = Sequential([
  Embedding(input_dim=5000, output_dim=16),
  LSTM(64),
  Dense(128, activation="relu"),
  Dense(64, activation="relu"),
  Dense(5, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
num_epochs = 100
history = model.fit(padded_train, label_train, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=1)

Epoch 1/100
26/26 [==============================] - 28s 972ms/step - loss: 1.6104 - accuracy: 0.1741 - val_loss: 1.6099 - val_accuracy: 0.2139
Epoch 2/100
26/26 [==============================] - 24s 941ms/step - loss: 1.6001 - accuracy: 0.2376 - val_loss: 1.6066 - val_accuracy: 0.2189
Epoch 3/100
26/26 [==============================] - 25s 969ms/step - loss: 1.5081 - accuracy: 0.3259 - val_loss: 1.6657 - val_accuracy: 0.1891
Epoch 4/100
26/26 [==============================] - 24s 936ms/step - loss: 1.2652 - accuracy: 0.3806 - val_loss: 1.8774 - val_accuracy: 0.2438
Epoch 5/100
26/26 [==============================] - 23s 887ms/step - loss: 1.0499 - accuracy: 0.4838 - val_loss: 2.3064 - val_accuracy: 0.1741
Epoch 6/100
26/26 [==============================] - 24s 908ms/step - loss: 0.9178 - accuracy: 0.5336 - val_loss: 2.5830 - val_accuracy: 0.1990
Epoch 7/100
26/26 [==============================] - 24s 934ms/step - loss: 0.8126 - accuracy: 0.6580 - val_loss: 2.8933 - val_accuracy: